In [ ]:
using LaTeXStrings, Colors, LinearAlgebra, Plots, Interact, Images,SymPy
import Plots.plot!

# Tarea 9: Algoritmos de Monte-Carlo



**Equipo:**

- 
- 

Esta tarea consiste de sólo 4 problemas que son una modificación no tan grande de lo que se vio en los videos. Son todos relativamente fáciles, pero requieren un tiempo no tan pequeño para hacerlos puesto que correr los programas ya no es cuestión de segundos, sino de minitos y cada prueba que se hace lleva entonces minutos, lo que típicamente implica horas de espera entre todas las pruebas. 

(2 puntos) Haz una función que calcule el área de una hiper-esfera de dimensión $N$ y radio $r$ usando integrales de Monte-Carlo. Para hacer esto usa ambos métodos: 

    (i) El método de los disparos.
    (ii) Usando la integral de la distribución de probabilidad de una variable aleatoria. 
    (iii) Finalmente compara tu resultado con el área que debería tener como función del número de puntos de Monte-Carlo.


### i

In [ ]:
# numero de puntos dentro Zn/numero de puntos n = P = Superficie esfera-n / Superficie-cubo-n 

d = 9 # dimensión
r = 2 # radio
n = 10^6 # Numero de pruebas
Zn = 0
x = zeros(d)
for i ∈ 1:n
    for j ∈ 1:d
        #x[j]= r*(1-rand())
        x[j] = 2*r*rand() - r
    end 
    Rsq = 0
    for j ∈ 1:d
       Rsq = Rsq + x[j]^2
    end
    R = sqrt(Rsq) # √(x_1^2 + x_2^2 + .... + x_9^2)
    if R <= r 
        Zn += 1 # Si cae dentro, aumenta en uno
    end
end
Volumen = (2*r)^d*Zn/n 
#Area = Volumen*d/(r^d)
Area = Volumen*d/r

println("el area de la hypersesfera de ", d, "dimensiones es ", Area)

### ii

In [ ]:
∫(g,a,b, n) = (b-a) * sum(g.(a.-(a-b)*rand(n)))/n

In [ ]:
d = 3
r = 1
x = zeros(d)

suma = sum([x[i]^2 for i in 1:d])

f(suma) = sqrt(suma)

In [ ]:
∫(g,a,b, n) = (b-a) * sum(g.(a.-(a-b)*rand(n)))/n

In [ ]:
@vars r ϕ θ
h = (r^2*sin(ϕ))

integrate(h, (r,0,2), (ϕ,0,π), (θ,0,2*π))

**1.3 pts (1.3 de 10)**

**Notas:** 
- Me parece que 
```julia
Area = Volumen*d/(r^d)
```
debería ser
```julia
Area = Volumen*d/r
```
Les recomiendo revisar [este link](https://en.wikipedia.org/wiki/Volume_of_an_n-ball#Approximation_for_high_dimensions).
- Sus dos incisos deberían estar dentro de una función.
- Su segundo inciso sólo funciona para tres dimensiones. Necesitan la diferencial de volumen para coordenadas esféricas $n$-dimensionales.
- Les faltó el tercer inciso. 

(4 puntos) Da una interpreatación de caminantes aleatorios a la siguiente ecuación diferencial parcial y resuelvela (para alguna condición a la frontera que dibujes y una función $f(x,y)$ que tú desidas) usando caminantes aleatorios: 

$$\nabla u = c$$ con $c$ una constante y $\nabla$ el laplaciano. 

In [ ]:
placa = load("placa.png")

In [ ]:
n,m = size(placa)
@show n*m
retícula = [placa[i,j] == placa[1,1] ? -1 : 0 for i in 1:n, j in 1:m]; 
heatmap(retícula)

In [ ]:
function posicion_permitida(i,j, retícula)
    if retícula[i,j] == -1 
        return false
    else
        return true
    end
end

In [ ]:
function evoluciona_caminante(i,j, retícula)
    n = rand([1,2,3,4])
    if n == 1 
        return i + 1, j
    elseif n == 2
        return i - 1, j
    elseif n == 3
        return i, j + 1
    elseif n == 4
        return i, j - 1
    end
end 

function trayectoria_hasta_salir!(i,j, retícula)
    test = true
    I = [i]
    J = [j]
    while test
        i,j = evoluciona_caminante(i,j, retícula)
        push!(I, i)
        push!(J, j)
        test = posicion_permitida(i,j, retícula)
    end
    return I, J
end 

In [ ]:
function minimum_positivo(matriz)
    n,m = size(matriz)
    x = Inf
    for i in 1:n, j in 1:m
        if matriz[i,j]<x && matriz[i,j] >= 0
            x = matriz[i,j]
        end
    end
    if x ≠ Inf
        return x
    else
        return 0
    end 
end

In [ ]:
function resuelve_Laplace(retícula, f, n, c)    
    contador = 0
    n1, n2 = size(retícula)
    solución = zeros(n1,n2)
    numero_de_caminantes = [retícula[i,j]<0 ? -1 : 0 for i in 1:n1, j in 1:n2]
    while minimum_positivo(numero_de_caminantes)<n
        for i in 1:n1, j in 1:n2
            contador += 1
            if mod(contador, 1500) == 0
                println("revisando vertice ($i ,$j)")
                println("numero maximo de caminantes = $(maximum(numero_de_caminantes))")
                println("numero mínimo de caminantes = $(minimum_positivo(numero_de_caminantes))")
                println(" ")
            end
            test = posicion_permitida(i,j, retícula)
            if test
                x,y = trayectoria_hasta_salir!(i,j,retícula)
                s = f(x[end], y[end]) + (n*m*c)
                for k in 1:length(x)-1
                    solución[x[k],y[k]] += s
                    numero_de_caminantes[x[k],y[k]] += 1
                end
            end
        end
    end
    return solución, numero_de_caminantes
end

In [ ]:
function mezcla_solucion_n(s, nc)  # Esta función simplemente sirve para hacer los promedios de la solución correctamente. 
    n,m = size(s)
    [nc[i,j] > 0 ? s[i,j]/nc[i,j] : NaN for i in 1:n, j in 1:m]
end

In [ ]:
f(y,x) = x^2+y
@time solución, numero_de_caminantes = resuelve_Laplace(retícula, f, 10, π)

In [ ]:
@show maximum(numero_de_caminantes), minimum_positivo(numero_de_caminantes)
ss = mezcla_solucion_n(solución, numero_de_caminantes);

In [ ]:
heatp = heatmap(numero_de_caminantes, aspect_ratio = :equal)
heatT = heatmap(ss, aspect_ratio = :equal)
plot(heatT, heatp)

*Ahora añadiremos la solción con una distribución más homogenea de los caminantes*

In [ ]:
function minimos_positivos(matriz)
    y = minimum_positivo(matriz)
    I = findall(x-> x == y, matriz);
end

function minimos_positivos(matriz, mini)
    I = findall(x-> x == mini, matriz)  
end

In [ ]:
function resuelve_Laplace_h(retícula, f, n, c)    
    contador = 0
    n1, n2 = size(retícula)
    solución = zeros(n1,n2)
    numero_de_caminantes = [retícula[i,j]<0 ? -1 : 0 for i in 1:n1, j in 1:n2]
    mini = 0
    I = minimos_positivos(numero_de_caminantes)
    while minimum_positivo(numero_de_caminantes)<n+1
        if mini == 0 && contador > 0
            I = minimos_positivos(numero_de_caminantes)
            mini = numero_de_caminantes[I[1]]
        elseif mini>0
            I = minimos_positivos(numero_de_caminantes, mini)
            if length(I) == 0
                I = minimos_positivos(numero_de_caminantes)
            end
            mini = numero_de_caminantes[I[1]]
        end
        j = rand(I)
        contador += 1
        if mod(contador, 5000) == 0
            println("revisando vertice ($(j[1]) ,$(j[2]))")
            println("numero maximo de caminantes = $(maximum(numero_de_caminantes))")
            println("numero mínimo de caminantes = $(minimum_positivo(numero_de_caminantes))")
            println(" ")
        end
        test = posicion_permitida(j[1],j[2], retícula)
        if test
            x,y = trayectoria_hasta_salir!(j[1],j[2],retícula)
            s = f(x[end], y[end]) + (n*m*c)
            for k in 1:length(x)-1
                solución[x[k],y[k]] += s
                numero_de_caminantes[x[k],y[k]] += 1
            end
        end
    
    end
    return solución, numero_de_caminantes
end

In [ ]:
dg = load("placa.png")
n,m = size(dg)
@show n*m
retícula = [dg[i,j] == dg[1,end] ? -1 : 0 for i in 1:n, j in 1:m]; 

In [ ]:
f(x,y)= x^2+y

In [ ]:
@time solución, numero_de_caminantes = resuelve_Laplace_h(retícula, f, 1000, π)

In [ ]:
sum(numero_de_caminantes)/631

In [ ]:
s_dg = mezcla_solucion_n(solución, numero_de_caminantes);
n,m = size(s_dg)
s_dgf = [s_dg[i,j] == 0 ? f(i,j) : s_dg[i,j] for i in 1:n, j in 1:m]

In [ ]:
heat_pdg = heatmap(numero_de_caminantes, aspect_ratio = :equal)
heat_Tdg = heatmap(s_dgf, aspect_ratio = :equal)
plot(heat_pdg, heat_Tdg)

*El colocar una condición de frontera a una región nos aseguea que una caminata aleatoria termine en algun punto de la frontera $(x',y')$, con*

$$u(x',y') = f(x',y')$$

*Uno puede estimar el valor de $u(x,y)$, realizando muchas caminatas aleatorias -las cuales comienzan en el punto $(x,y)$- como el promedio sobre las N caminatas,así,*

$$u(x,y) \sim \frac{1}{N} \sum f(x'_{i},y'_{i})$$

**3.5 pts (4.8 de 10)**

**Notas:** 
- Están en lo correcto en el sentido de que el único cambio que había que hacer al notebook del profesor era
```julia
s = f(x[end], y[end]) + (n*m*c)
```
Sin embargo, su término extra es incorrecto. Si hacen el desarrollo de diferencias finitas, su término extra debería ser proporcional a $-c/4$. 
- La imagen que me mandaron no es óptima para este problema, lo ideal es que su imagen sólo tenga dos colores y la que me mandaron no cumple con eso. 

(3 puntos) Simula un sistema de 50 discos suaves en una caja (de lado $1$). Para esto considera un potencial de interacción $U(r_{i,j})$ que depende de $r_{i,j}$, la distancia entre el disco $i$ y el disco $j$ ($i \ne j$) igual al potencial de Lennard Jones  ($U(r_{i,j}) = \left(\frac{\sigma}{r_{i,j}}\right)^{12} - \left(\frac{\sigma}{r_{i,j}}\right)^6$, donde $\sigma$ es el diametro de los discos) si $r_{i,j} \le \sigma$ y $U(r_{i,j}) = 0$ cuando $r_{i,j} > \sigma$.

Tu simulación hazla similar a lo que se hizo en los videos. Comienza con discos de radio $0$ y crece el tamaño de esos discos hasta que se llegue a una fracción de empaquetamiento alta. Nota que esta vez la simulación sí depende de la temperatura (prueba con unas 3 o 4 temperaturas diferentes). 

Nota: La fracción de empaquetamiento es la suma del área de todos los discos dividido entre el área de la caja. 

In [ ]:
mutable struct Disco #Creamos el objeto 'disco'
    radio::Real
    posicion::Vector
    color
end

In [ ]:
#Creamos un metodo para graficar un disco
function plot!(disco::Disco)
    θ = 0:0.05:(2π+0.05)
    x = disco.radio*(cos.(θ)).+disco.posicion[1]
    y = disco.radio*(sin.(θ)).+disco.posicion[2]
    plot!(x,y,fill=true,color = disco.color)
end

In [ ]:
function overlap(discos) #Funcion para ver si se solapan discos
    n = length(discos)
    for i in 1:n-1
        for j in i+1:n
            if norm(discos[i].posicion - discos[j].posicion) < discos[i].radio+discos[j].radio
                return true
            end
        end
    end
    
    for i in 1:n
        if discos[1].posicion[1] < discos[i].radio || discos[i].posicion[1] > 15-discos[i].radio || discos[i].posicion[2] < discos[i].radio || discos[1].posicion[2] > 16-discos[i].radio
            return true
        end
    end
    return false
    
end

function overlap(discos,i) #Funcion para ver si se solapa un disco con sus vecinos
    disco = discos[i]
    n = length(discos)
    if disco.posicion[1]<disco.radio || disco.posicion[1] > 15-disco.radio || disco.posicion[2] < disco.radio || disco.posicion[2] > 15-disco.radio
        return true
    end
    
    for j in (1:(i-1))∪((i+1):length(discos))
        if norm(discos[j].posicion-disco.posicion) < discos[j].radio + disco.radio
            return true
        end
    end
    return false
end

In [ ]:
discos = []
for i in 1:50
    d = Disco(0., rand(0:0.01:1,2), RGB(0.8,0.5,0.2))
        push!(discos,d)
end
length(discos)

In [ ]:
function PLJ(x1,x2,σ) #Funcion para el potencial de Lennard-Jones
    if norm(x2-x1) > σ || norm(x2-x1) == 0
        return 0.0
    else 
        return (σ/norm(x2-x1))^12 -(σ/norm(x2-x1) )^6
    end  
end

In [ ]:
function Distribucion_Boltzmann(Δ𝐻, 𝑇) #Función para la distribución de Boltzmann
    if Δ𝐻 == 0
        return 1
    else
        return min(1,exp(-Δ𝐻/𝑇))
    end
end

In [ ]:
function simulacion_LJ!(discos, T, pasos = length(discos)*100, δ=0.05)
    
    nd = length(discos) #No de discos
    
    #Copia de los discos que usaremos para el algoritmo
    discos2=deepcopy(discos)
    D = [discos2]
    for i in 1:pasos
        
        if mod(i,5000) == 0
            println(i*100/pasos, "%")
        end
        
        discos2 = deepcopy(discos)
        #Tomamos dos discos aleatorios y obtenemos el potencial de Lennard-Jones asociado
        j = rand(1:nd)
        l = rand(1:nd)
        σ = 2*discos2[j].radio #Diametro de un disco
        ΔH = PLJ(discos2[j].posicion, discos2[l].posicion,σ) #Calculo de la energía potencial
        
        test = rand(0:0.01:1) #Generamos un numero aleatorio y lo comparamos con la distribución de Boltzmann asociado a la energía
        if test < Distribucion_Boltzmann(ΔH,T) 
            for k in 1:nd
                discos2[k].radio += 0.01
            end
            
            if overlap(discos2) #Si los discos se solapan entonces omitmos 
                
            else
                discos = deepcopy(discos2)
                push!(D, deepcopy(discos2) )
            end
            continue
        end
        #Modificamos de manera aleatoria algún disco
        θ = rand()*2*π
        x = δ*[cos(θ), sin(θ)]
        discos2[j].posicion .+= x
        
        #Vemos si existe algún solape entre el disco modificado y alrededores
        if overlap(discos2,j)
            continue
        else
            discos[j] = deepcopy(discos2[j])
            push!(D, deepcopy(discos) )
        end
    end
    return D
end

In [ ]:
discos = []
for i in 1:50
    d = Disco(0., rand(0:0.1:1, 2), RGB(0.1, 0.8,0.4) )
    push!(discos, d)
end

length(discos)

#Caso 1: T=1
@time D = simulacion_LJ!(discos, 1, 2500000);
discos = deepcopy(D[end]);

In [ ]:
using Plots
discos_suaves_1 = @animate for i in 1:100:length(D)
    plot(aspect_ratio = :equal, key = false, xlim = (0.0,1.0 ), ylim = (0.0,1.0) )
    plot!([0,1,1,0,0], [0,0,1,1,0], color = RGB(0.1, 0.5, 0.7) )
    for j in 1:length(D[1])
        plot!(D[i][j])
    end
    plot!(show = :ijulia, title = "ϕ = $(round(100*π*D[i][1].radio^2/15^2 , digits = 3) )")
end

In [ ]:
gif(discos_suaves_1, "Discos_duros.gif", fps = 24)

In [ ]:
discos = []
for i in 1:50
    d = Disco(0., rand(0:0.1:1, 2), RGB(0.1, 0.8,0.4) )
    push!(discos, d)
end

length(discos)

#Caso 2: T=100
@time D = simulacion_LJ!(discos, 100, 2500000);
discos = deepcopy(D[end]);

In [ ]:
discos_suaves_2 = @animate for i in 1:100:length(D)
    plot(aspect_ratio = :equal, key = false, xlim = (0.0,1.0 ), ylim = (0.0,1.0) )
    plot!([0,1,1,0,0], [0,0,1,1,0], color = RGB(0.1, 0.5, 0.7) )
    for j in 1:length(D[1])
        plot!(D[i][j])
    end
    plot!(show = :ijulia, title = "ϕ = $(round(100*π*D[i][1].radio^2/15^2 , digits = 3) )")
end

In [ ]:
gif(discos_suaves_2, "Discos_duros.gif", fps = 24)

In [ ]:
discos = []
for i in 1:50
    d = Disco(0., rand(0:0.1:1, 2), RGB(0.1, 0.8,0.4) )
    push!(discos, d)
end

length(discos)

#Caso 3: T=25
@time D = simulacion_LJ!(discos, 25, 2500000);
discos = deepcopy(D[end]);

In [ ]:
discos_suaves_3 = @animate for i in 1:100:length(D)
    plot(aspect_ratio = :equal, key = false, xlim = (0.0,1.0 ), ylim = (0.0,1.0) )
    plot!([0,1,1,0,0], [0,0,1,1,0], color = RGB(0.6, 0.1, 0.1) )
    for j in 1:length(D[1])
        plot!(D[i][j])
    end
    plot!(show = :ijulia, title = "ϕ = $(round(100*π*D[i][1].radio^2/15^2 , digits = 3) )")
end

In [ ]:
gif(discos_suaves_3, "Discos_duros.gif", fps = 24)

**2.5 pts (7.3 de 10)**

**Notas:**
- Su función `overlap` es inconsistente, pues considera una caja de lados `15×16`, pero quieren simular una caja de lados `1×1`. 
- En realidad están simulando discos duros, el error está en los if
```julia
if overlap(discos2,j)
    continue
else
    discos[j] = deepcopy(discos2[j])
    push!(D, deepcopy(discos) )
end
```
Para discos suaves, hay una probabilidad distinta de cero de que el sistema avance de estado aún si hay `overlap`.

(3 puntos) Simula el modelo de Ising para una red hexagonal, es decir, cada vértice tiene 3 vecinos (y si dibujas la red, verás hexágonos). Para esto necesitas hacer 2 cambios importantes a lo visto en los videos: 
    
    (i) Modificar la función que produce los vecinos. 
    (ii) Modificar la función que grafica para que cada nodo lo ponga como un hexágono de un color si el espín apunta hacia arriba y de otro si apunta hacia abajo. 
    
    Revisa en particular los caso $J = -1/2$ y $J = 1/2$ y de este último obtén (haciendo un promedio de 5 casos nada más) la magnetización como función de la temperatura. 

In [ ]:
function jj(i, n, m)
    ((i-1)÷n) % m
end
    
function vecinos(n, m)
    vec = []
    for i in 1:n*m
        j = jj(i, n, m)
        k = mod1(j*n, 4)
        l = mod1((j+1)*n, 4)
        if k == 1
            if mod1(i, 2) == 0
                push!(vec, [i,i,i])
            else
                push!(vec, [mod1(i-1, n)+l, mod1(i+1, n)+l, mod1(i, n)+(k-1)*n])
            end
        elseif k == 2
            if mod1(i, 2) == 1
                push!(vec, [i,i,i])
            else
                push!(vec, [mod1(i-1, n)+(k-1)*n, mod1(i+1, n)+(k-1)*n, mod1(i, n)+l])
            end
        elseif k == 3
            if mod1(i, 2) == 1
                push!(vec, [i,i,i])
            else
                push!(vec, [mod1(i-1, n)+l, mod1(i+1, n)+l, mod1(i, n)+(k-1)*n])
            end
        else
            if mod1(i, 2) == 0
                push!(vec, [i,i,i])
            else
                push!(vec, [mod1(i-1, n)+(k-1)*n, mod1(i+1, n)+(k-1)*n, mod1(i, n)+l])
            end
        end
    end
    return vec
end

In [ ]:
function suma(σ, vecinos1) 
    s = 0
    for i in 1:length(σ)
        s += sum(σ[vecinos1[i]])*σ[i]
    end
    return s
end
H(σ, vecinos1; h = 0, J = 1) = -h*sum(σ) - J*suma(σ, vecinos1) 

In [ ]:
function ΔH(σ, i, vecinos1; h = 0, J = -1)
    s = 2h*σ[i] 
    s += 4J*σ[i]*sum(σ[vecinos1[i]])
    return s
end

In [ ]:
function Boltzmann(Δ𝐻, 𝑘𝑇) 
    if Δ𝐻 == 0
        return 1
    else
        return min(1,exp(-Δ𝐻/𝑘𝑇))
    end
end
function Metropolis_ising!(σ, vecinos1, n, m, H1; n_pasos = 1, kT = 1, kargs...)
    l = length(σ)
    centro_material = [i*n+1+j for i in 1:m-2, j in 1:n-2]
    mag = 0
    for i in 1:n_pasos
        j = rand(centro_material)
        ΔH1 = ΔH(σ, j, vecinos1; kargs...)
        p = Boltzmann(ΔH1, kT)
        x = rand()
        if x<p
            σ[j] *= -1
        end
        if i > n_pasos-1000
            mag += sum(σ)/l
        end
        H1 += ΔH1
    end
    return σ, H1, mag/1000
end

In [ ]:
function plot_sigma(σ, n, m, color = RGB(1,1,1))
    
     heatmap(reshape(σ, m,n), color = [RGB(0,0,0),RGB(1,0,0), color], aspect_ratio = 1, cbar = false, grid = true, axis = false, ticks = false)
end
plot_sigma(σ, n, m, color = RGB(1,1,1)) = heatmap(reshape(σ, m,n), color = [RGB(0,0,0), color], aspect_ratio = 1, cbar = false, grid = false, axis = false, ticks = false)

In [ ]:
function plot_final(σ, n, time, magnetización, color, t)
    plot_sigma(σ, n, n, color)
    plot1 = plot!(title = "$(time[end])")
    plot2 = plot(time, abs.(magnetización), key = false, xlim = (0,t), ylim = (0,1), 
                 title = "magnetización = $(round(magnetización[end], digits = 3))")
    plot(plot1, plot2)
end

function animacion_interactiva(L, kT, J, h, t, color)
    vecinosnm = vecinos(L, L)
    σ = rand([-1, 1], L^2)
    σ[1:L] .= ones(L)
    σ[1:L:(L-1)*L+1] .= ones(L)
    σ[L:L:L^2] .= ones(L)
    σ[L*(L-1)+1:L^2] .= ones(L)
    H1 = H(σ, vecinosnm)
    magnetización = []
    
    @gif for i in 1:t
        time = 1:i
        σ, H1, mag = Metropolis_ising!(σ, vecinosnm, L, L, H1, n_pasos = 10L^2, kT = kT, h = h, J = J)
        push!(magnetización, mag)
        plot_final(σ, L, time, magnetización, color, t)
    end
end

function np(n, m)
    return n
end

In [ ]:
boton_L = textbox(value = "5")
barra_h = slider(0:0.01:2, label = "h", value = 0)
barra_kT = slider(0:0.01:8, label = "kT", value = 0)
barra_t = textbox(value = "1")
barra_J = slider(-1:0.02:1, label = "J", value = 0.5)
boton_r = button("reset")
colores = colorpicker(value = RGB(0.3, 0.7, 0.9))
botones = OrderedDict(:boton_L => boton_L, :barra_h => barra_h, :barra_kT => barra_kT, :boton_r => boton_r, 
                      :barra_J => barra_J, :barra_t => barra_t, :colores => colores)
w = Interact.Widget{:mi_widget}(botones)

layout = vbox(
    hbox(
        vbox(
            hbox(latex("L = "), boton_L),
            boton_r,
            colores
        ),
        vbox(
            barra_h,
            barra_kT,
            barra_J
        )
    ),
    vbox(
        hbox(
            "tiempo de animación = ",
            barra_t
        )
    )
)
@layout! w layout

In [ ]:
gr()
animacion = map(animacion_interactiva, 
    map(np, map(x-> parse(Int, x), botones[:boton_L]), botones[:boton_r]), 
    botones[:barra_kT], 
    botones[:barra_J],
    botones[:barra_h],
    map(x-> parse(Int, x), botones[:barra_t]),
    botones[:colores]);

In [ ]:
hbox(animacion, w)

In [ ]:
MAG = []

In [ ]:
m,n = 100,100
for caso in 1:5 #Modificamos el caso 
    if mod(caso, 3) == 0
        @show caso
    end
    vecinosnm = vecinos(n,m)
    σ = rand([-1,1], n*m)
    σ = rand([-1,1], n*m)
    σ[1:n] = ones(n)
    σ[1:n:(n-1)*m+1] = ones(m)
    σ[n:n:n*m] = ones(m)
    σ[n*(m-1)+1:n*m] = ones(n)
    H1 = H(σ, vecinosnm)
    kTf = 6
    Metropolis_ising!(σ, vecinosnm, n,m, H1, n_pasos = 1500*n*m, kT = 0, h = 0.0, J = 1/2)
    magnetización = []
    for kT in 0:0.01:kTf
        σ, H1, mag = Metropolis_ising!(σ, vecinosnm, n,m, H1, n_pasos = 200*n*m, kT = kT, h = 0.0, J = 1/2)
        push!(magnetización, mag)   
    end
    push!(MAG, magnetización)
end

In [ ]:
mp = [sum([MAG[j][i] for j in 1:length(MAG)]) for i in 1:length(MAG[1])]

In [ ]:
x = 0:0.01:6
default(fontfamily = "sans-serif")
theme(:dark)

plot(x, mp./30, color = :red, label = false, xlabel = "kT", ylabel = "m", title = "kT vs m")

**2 pts (9.3 de 10)**

**Nota:** Revisen su función `vecinos`, ya que no devuelve una malla hexagonal. Para comprobarlo, hagan a mano un caso sencillo (e.g. $L = 5$) y contrástenlo con los resultados de su función. 